In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten,Dropout
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


In [ ]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

169009152/169001437 [==============================] - 12s 0us/step


In [ ]:
aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [ ]:
shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a) 
a=layers.Dropout(0.2)(a)  #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
a=layers.Dropout(0.2)(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

In [ ]:
sgd = SGD(lr=0.01, momentum=0.91,clipvalue=0.6)


model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])
call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=5,restore_best_weights=True)
checkpoint=ModelCheckpoint('RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [ ]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=80,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/80
391/390 [==============================] - ETA: 0s - loss: 4.3695 - accuracy: 0.0395
Epoch 00001: val_accuracy improved from -inf to 0.07770, saving model to RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5
391/390 [==============================] - 371s 950ms/step - loss: 4.3695 - accuracy: 0.0395 - val_loss: 4.1180 - val_accuracy: 0.0777
Epoch 2/80
391/390 [==============================] - ETA: 0s - loss: 4.0191 - accuracy: 0.0837
Epoch 00002: val_accuracy improved from 0.07770 to 0.13920, saving model to RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5
391/390 [==============================] - 367s 939ms/step - loss: 4.0191 - accuracy: 0.0837 - val_loss: 3.6849 - val_accuracy: 0.1392
Epoch 3/80
391/390 [==============================] - ETA: 0s - loss: 3.7202 - accuracy: 0.1286
Epoch 00003: val_accuracy improved from 0.13920 to 0.15950, saving model to RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5
391/390 [===

In [8]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [11]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score

In [15]:
print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test,y_pred))
print(precision_score(Y_test,y_pred,average="weighted"))
print(recall_score(Y_test,y_pred,average="weighted"))

              precision    recall  f1-score   support

           0       0.61      0.89      0.72       100
           1       0.65      0.71      0.68       100
           2       0.49      0.41      0.45       100
           3       0.38      0.30      0.34       100
           4       0.45      0.45      0.45       100
           5       0.56      0.42      0.48       100
           6       0.75      0.56      0.64       100
           7       0.57      0.65      0.60       100
           8       0.58      0.73      0.65       100
           9       0.75      0.70      0.73       100
          10       0.39      0.36      0.37       100
          11       0.45      0.47      0.46       100
          12       0.61      0.62      0.62       100
          13       0.50      0.56      0.53       100
          14       0.53      0.45      0.49       100
          15       0.43      0.57      0.49       100
          16       0.56      0.53      0.54       100
          17       0.65    

In [1]:
#RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten,Dropout
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a) 
a=layers.Dropout(0.2)(a)  #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Dropout(0.2)(a)
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Dropout(0.2)(a)
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Dropout(0.2)(a)
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
a=layers.Dropout(0.2)(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

model.load_weights('../weights/RESNET_SGD_WITH_DROPOUT_REGULAIZATION.hdf5')


y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score

print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test,y_pred))
print(precision_score(Y_test,y_pred,average="weighted"))
print(recall_score(Y_test,y_pred,average="weighted"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
              precision    recall  f1-score   support

           0       0.78      0.77      0.77       100
           1       0.94      0.48      0.64       100
           2       0.50      0.52      0.51       100
           3       0.44      0.21      0.28       100
           4       0.50      0.37      0.43       100
           5       0.45      0.55      0.50       100
           6       0.75      0.70      0.73       100
           7       0.54      0.54      0.54       100
           8       0.57      0.80      0.66       100
           9       0.79      0.64      0.71       100
          10       0.34      0.39      0.36       100
          11       0.47      0.47      0.47       100
          12       0.64      0.61      0.63       100
          13       0.65      0.47      0.55       100
          14       0.51      0.55      0.53       100
          15     